In [1]:
import requests
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
tqdm.pandas()

Clean DOI

In [2]:
path='../IPCC_bibliography/AR6/structured_data/'

In [3]:
#data for each wg
df_wgi = pd.read_json(path+'data_wg1.jsonl', lines= True)
df_wgii = pd.read_json(path+'data_wg2.jsonl', lines= True)
df_cross_wgii = pd.read_json(path+'data_cross_wg2.jsonl', lines= True)
df_wgiii = pd.read_json(path+'data_wg3.jsonl', lines= True) 

In [4]:
#all data
df_ipcc=pd.concat([df_wgi, df_wgii, df_cross_wgii, df_wgiii], ignore_index=True)

In [5]:
def get_doi_cleaned(x):
    low_x=str(x).lower()
    if pd.isna(x):
        return None
    if low_x.find('https://doi.org/')>0:
        return low_x.replace('https://doi.org/','')
    else:
        return low_x

In [6]:
df_ipcc['doi']=df_ipcc.loc[:,'doi'].apply(lambda x: get_doi_cleaned(x))

Get open_access, sdg, countries from OpenAlex

In [7]:
def aplatir(conteneurs):
    return [conteneurs[i][j] for i in range(len(conteneurs))for j in range(len(conteneurs[i]))]

In [8]:
json_OA=[]
def get_open_alex_data(row):
    doi=row.doi
    if pd.isna(doi)==False:
        url=f"https://api.openalex.org/works?filter=doi:{doi}"
        response = requests.get(url)
        data = response.json()
        if 'results' in data.keys():
            return {"doi": doi, "results": data.get('results')}
        else:
            return {"doi": doi, "results": []}

In [9]:
def get_status_sdg_coutries(row):
    doi=row.doi
    data=pd.read_json('data_OpenAlex.json')
    if 'results' not in data.keys():
        return None,None,None
    if data.get('results')!=[]:
        open_access=data.get('results')[0].get('open_access')
        if open_access!={}:
            status={'is_oa':open_access.get('is_oa'),'oa_status':open_access.get('oa_status')}
        else:
            status=None

        sdgs=data.get('results')[0].get('sustainable_development_goals')
        if sdgs!=[]:
            sdgs_id_name=[{'id': str(sdg.get('id'))[-2:].replace("/",""), 'name': sdg.get('display_name')} for sdg in sdgs]
        else:
            sdgs_id_name=None

        authors=data.get('results')[0].get('authorships')
        if authors!=[]:
            countries=list(set(aplatir([author.get('countries') for author in authors]))) 
        else:
            countries=None
    else:
        return None,None,None

    return status, sdgs_id_name, countries

In [10]:
df_unique_doi=df_ipcc.drop_duplicates(subset=['doi'])

In [11]:
df_unique_doi.loc[:,'doi']=df_unique_doi.loc[:,'doi'].apply(lambda x: None if str(x)[:4]=='http' else x)

In [12]:
df_unique_doi=df_unique_doi.dropna(subset=['doi'])

In [13]:
df_unique_doi.progress_apply(get_open_alex_data, axis= 1)
print(json_OA)

  0%|          | 0/53050 [00:00<?, ?it/s]

In [ ]:
json_OA.to_json(path+'data_OpenAlex.json', orient='records')

In [ ]:
status_list = []
sdg_list = []
countries_list = []
for i,row in df_ipcc.iterrows():
    print(i)
    status, sdg, countries=get_status_sdg_coutries(row)

    status_list.append(status)
    sdg_list.append(sdg)
    countries_list.append(countries)

df_ipcc['status'] = status_list
df_ipcc['sdg'] = sdg_list
df_ipcc['countries'] = countries_list

In [ ]:
data_counts = aplatir(list(df_ipcc['countries'])).value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
plt.pie(data_counts, labels=data_counts.index, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
plt.show()